Instale las bibliotecas necesarias para realizar el web scraping. La biblioteca más popular en Python para esto es requests para obtener páginas web y BeautifulSoup o Scrapy para analizar el HTML.

In [1]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/5c/f2/f3faa20684729d3910af2ee142e30432c7a46a817eadeeab87366ed87bbb/charset_normalizer-3.2.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
   ---------------------------------------- 62.6/62.6 kB 3

Y a su vez BeautifulSoup4, esta biblioteca ayuda a analizar el HTML y extraer datos de manera más fácil y estructurada.

In [2]:
pip install beautifulsoup4

     -------------------------------------- 143.0/143.0 kB 4.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


De la seccion Top 100 de la pagina extraigo los titulos y URL de cada libro, les coloco Id segun su posicion en el top

In [36]:
import requests
from bs4 import BeautifulSoup

# URL del top 100 de libros
url = 'https://www.cuspide.com/cienmasvendidos'

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar si la solicitud fue exitosa 
# (código 200 esto significa OK, es decir que la solicitud fue procesada correctamente por el servidor)
if response.status_code == 200:
    # Analizar el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar todos los elementos <article>
    articulos = soup.find_all('article')

    # Extraer los datos de los libros de los elementos <article>
    libros = []
    # id = 0
    for articulo in articulos:
        titulo = articulo.find('a')['title']
        url = articulo.find('a')['href']
        # id += 1
        
        # Creo un diccionario que en cada ciclo se actualiza con los datos de los libros
        libro = {
            # 'id' : id,
            'titulo': titulo,
            'url' : 'https://www.cuspide.com/' + url
        }
        # Cada vez que se actualice el diccionario este se va a agregar a la lista, llenandose asi de todos los datos de los libros
        libros.append(libro)
        
    # Mostrar los datos de los libros
    for libro in libros:
        # print(f"Id: {libro['id']}")
        print(f"Título: {libro['titulo']}")
        print(f"URL: {libro['url']}")

else:
    print('Error al acceder a la página:', response.status_code)


Título: Una Familia Anormal
URL: https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal
Título: Este Dolor No Es Mio
URL: https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio
Título: Destroza Este Diario ( A Todo Color )
URL: https://www.cuspide.com//Libro/9789501296808/Destroza+Este+Diario+(+A+Todo+Color+)
Título: Gaturro Messi
URL: https://www.cuspide.com//Libro/9789878151885/Gaturro+Messi
Título: La Scaloneta
URL: https://www.cuspide.com//Libro/9789878151632/La+Scaloneta
Título: Messi , Campeon Del Mundo
URL: https://www.cuspide.com//Libro/9789878151335/Messi+++Campeon+Del+Mundo
Título: El Viento Conoce Mi Nombre
URL: https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+Mi+Nombre
Título: Dibu Martinez : Pasion Por El Futbol
URL: https://www.cuspide.com//Libro/9789878940182/Dibu+Martinez+++Pasion+Por+El+Futbol
Título: Pato Horneado Y El Rescate En La Jungla
URL: https://www.cuspide.com//Libro/9789508701985/Pato+Horneado+Y+El+Rescate+En+La+Jungla
Título

De cada libro en especifico extraigo su precio en pesos y dolares, como figuran en la pagina

In [37]:
#Funcion obtener_precios desfazada para comprobar el funcionamiento y lo que retorna ars_span
urlp = 'https://www.cuspide.com/Libro/9789877364262/Una+Familia+Anormal'
response = requests.get(urlp)
response.status_code == 200
soup = BeautifulSoup(response.text, 'html.parser')
precio_container = soup.find('div', class_='md-precio')
ars_span = precio_container.find('span', string='AR$').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.')
print(ars_span)

4199.00


In [38]:
import requests
from bs4 import BeautifulSoup

def obtener_precios(urlp):
    response = requests.get(urlp)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        precio_container = soup.find('div', class_='md-precio') # Encuentra el div con la clase md-precio
        if precio_container:
            # En AR$ se necesitan dos replace para sacar los puntos como separador de miles y reemplazar las comas por puntos como separadores de decimales
            ars_span = precio_container.find('span', string='AR$').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.') # Encuentra el span que contenga AR$ y selecciona su hermano siguiente, que serian los digitos del precio
            #En U$s solo con colocar el replace de comas por puntos como separadores de miles bastaria porque no hay libros que lleguen a la unidad de miles, pero si en algun momento llega a ese monto no va a tener que modificarse nada
            usd_span = precio_container.find('span', string='U$s').next_sibling.get_text(strip = True).replace('.', '').replace(',', '.') 
        return ars_span, usd_span
    return "Precio no encontrado", "Precio no encontrado"

# Obtener y mostrar precios
precios = []
for librop in libros: #Utilizo la lista de libros creada anteriormente para iterar los url y asi extraer el precio de cada uno de forma automática
    nombre = librop['titulo']
    urlp = librop['url']
    ars_precio, usd_precio = obtener_precios(urlp)

    # Creo un diccionario que en cada ciclo se actualiza con los precios de los libros
    precio = {
        'AR$' : ars_precio,
        'U$s' : usd_precio
    }
    # En cada ciclo los datos dentro del diccionario se van a agregar a la lista, llenandose asi de todos los precios de los libros
    precios.append(precio)
    
    print(f"Libro: {nombre}")
    print(f"Precio en AR$: {precio['AR$']}")
    print(f"Precio en U$s: {precio['U$s']}")


Libro: Una Familia Anormal
Precio en AR$: 4199.00
Precio en U$s: 11.49
Libro: Este Dolor No Es Mio
Precio en AR$: 9900.00
Precio en U$s: 27.09
Libro: Destroza Este Diario ( A Todo Color )
Precio en AR$: 8500.00
Precio en U$s: 23.26
Libro: Gaturro Messi
Precio en AR$: 3690.00
Precio en U$s: 10.10
Libro: La Scaloneta
Precio en AR$: 5690.00
Precio en U$s: 15.57
Libro: Messi , Campeon Del Mundo
Precio en AR$: 4990.00
Precio en U$s: 13.65
Libro: El Viento Conoce Mi Nombre
Precio en AR$: 9999.00
Precio en U$s: 27.36
Libro: Dibu Martinez : Pasion Por El Futbol
Precio en AR$: 4399.00
Precio en U$s: 12.04
Libro: Pato Horneado Y El Rescate En La Jungla
Precio en AR$: 5500.00
Precio en U$s: 15.05
Libro: Born Y Quieto
Precio en AR$: 8900.00
Precio en U$s: 24.35
Libro: Este Invierno
Precio en AR$: 7399.00
Precio en U$s: 20.24
Libro: Las Aventuras De Lyna Y Dani : Elige Tu Historia
Precio en AR$: 3299.00
Precio en U$s: 9.03
Libro: Los Vestidos De Lena
Precio en AR$: 7999.00
Precio en U$s: 21.89
Libr

Combino la lista de libros con la de precios para que me quede una sola lista con esos datos juntos

In [41]:
libros_precios = []

for dicc in range(len(libros)):
    dicc_combinado = {**libros[dicc], **precios[dicc]}
    libros_precios.append(dicc_combinado)

print(libros_precios)


[{'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49'}, {'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09'}, {'titulo': 'Destroza Este Diario ( A Todo Color )', 'url': 'https://www.cuspide.com//Libro/9789501296808/Destroza+Este+Diario+(+A+Todo+Color+)', 'AR$': '8500.00', 'U$s': '23.26'}, {'titulo': 'Gaturro Messi', 'url': 'https://www.cuspide.com//Libro/9789878151885/Gaturro+Messi', 'AR$': '3690.00', 'U$s': '10.10'}, {'titulo': 'La Scaloneta', 'url': 'https://www.cuspide.com//Libro/9789878151632/La+Scaloneta', 'AR$': '5690.00', 'U$s': '15.57'}, {'titulo': 'Messi , Campeon Del Mundo', 'url': 'https://www.cuspide.com//Libro/9789878151335/Messi+++Campeon+Del+Mundo', 'AR$': '4990.00', 'U$s': '13.65'}, {'titulo': 'El Viento Conoce Mi Nombre', 'url': 'https://www.cuspide.com//Libro/9789500769013/El+Viento+Conoce+

Separo los libros que tienen precios con los que no se pudieron scrapear

In [44]:
import datetime

errores = []
for i in libros_precios:
    nombre = i['titulo']
    urli = i['url']
    precio_pesos = i['AR$']
    precio_dolares = i['U$s']
    fecha = datetime.date.today()
    error = {
            'titulo':i['titulo'],
            'url':i['url'],
            'fecha':fecha.strftime("%Y-%m-%d")
        }
    if (precio_pesos == 'Precio no encontrado') and (precio_dolares == 'Precio no encontrado'):
        errores.append(error)
        #ver como eliminar un diccionario que esta dentro de una lista
    else:
        continue
    

In [45]:
print(errores)

[]


Web scraping del valor del dolar blue, con un ciclo en el que calcule el precio al dolar blue de cada libro y  la fecha actual, cada vez que se ejecute y acutalice va a decir la fecha en la que esto se hizo

In [47]:
import requests
from bs4 import BeautifulSoup
import re # Esta libreria me ayuda a eliminar los strings que retorne la busqueda, devolviendome solo el valor numerico del dolar
import datetime # Esta libreria es para trabajar con fechas y horas

urld = 'https://dolarhoy.com/'
responsed = requests.get(urld)
responsed.status_code == 200
soupd = BeautifulSoup(responsed.text, 'html.parser')
dolar_signo = soupd.find('div', class_='venta').get_text(strip=True).replace('.', '').replace(',', '.') # Esto retorna el precio pero con la palabra Venta y el signo $(peso)
dolar = float(re.search(r'\d+', dolar_signo).group()) # Variable que limpia el valor para que retorne solo los numeros y lo convierte en flotante
print('Valor dolar blue:',dolar)

precios_blue = []
for librod in libros_precios:
    preciopesos = librod['AR$']
    fecha_actual = datetime.date.today()

    precioblue = {
        'U$s_blue' : round((float(preciopesos) / dolar),2),
        'Fecha' : fecha_actual.strftime("%Y-%m-%d")
    }
    precios_blue.append(precioblue)
print(precios_blue)

Valor dolar blue: 725.0
[{'U$s_blue': 5.79, 'Fecha': '2023-08-24'}, {'U$s_blue': 13.66, 'Fecha': '2023-08-24'}, {'U$s_blue': 11.72, 'Fecha': '2023-08-24'}, {'U$s_blue': 5.09, 'Fecha': '2023-08-24'}, {'U$s_blue': 7.85, 'Fecha': '2023-08-24'}, {'U$s_blue': 6.88, 'Fecha': '2023-08-24'}, {'U$s_blue': 13.79, 'Fecha': '2023-08-24'}, {'U$s_blue': 6.07, 'Fecha': '2023-08-24'}, {'U$s_blue': 7.59, 'Fecha': '2023-08-24'}, {'U$s_blue': 12.28, 'Fecha': '2023-08-24'}, {'U$s_blue': 10.21, 'Fecha': '2023-08-24'}, {'U$s_blue': 4.55, 'Fecha': '2023-08-24'}, {'U$s_blue': 11.03, 'Fecha': '2023-08-24'}, {'U$s_blue': 5.17, 'Fecha': '2023-08-24'}, {'U$s_blue': 11.03, 'Fecha': '2023-08-24'}, {'U$s_blue': 10.34, 'Fecha': '2023-08-24'}, {'U$s_blue': 8.97, 'Fecha': '2023-08-24'}, {'U$s_blue': 12.41, 'Fecha': '2023-08-24'}, {'U$s_blue': 9.52, 'Fecha': '2023-08-24'}, {'U$s_blue': 14.21, 'Fecha': '2023-08-24'}, {'U$s_blue': 6.76, 'Fecha': '2023-08-24'}, {'U$s_blue': 9.31, 'Fecha': '2023-08-24'}, {'U$s_blue': 5.38, 

Combino los diccionarios de los titulos y precios (AR$ y U$s) con el diccionario que contiene los precios en dolar blue

In [48]:
# Lista con todos los datos necesarios de cada libro

libros_completo = []

for dicclibros_precios, diccprecios_blue in zip(libros_precios, precios_blue):
    dicclibros_precios.update(diccprecios_blue)
    libros_completo.append(dicclibros_precios)

print(libros_completo)

[{'titulo': 'Una Familia Anormal', 'url': 'https://www.cuspide.com//Libro/9789877364262/Una+Familia+Anormal', 'AR$': '4199.00', 'U$s': '11.49', 'U$s_blue': 5.79, 'Fecha': '2023-08-24'}, {'titulo': 'Este Dolor No Es Mio', 'url': 'https://www.cuspide.com//Libro/9788484456810/Este+Dolor+No+Es+Mio', 'AR$': '9900.00', 'U$s': '27.09', 'U$s_blue': 13.66, 'Fecha': '2023-08-24'}, {'titulo': 'Destroza Este Diario ( A Todo Color )', 'url': 'https://www.cuspide.com//Libro/9789501296808/Destroza+Este+Diario+(+A+Todo+Color+)', 'AR$': '8500.00', 'U$s': '23.26', 'U$s_blue': 11.72, 'Fecha': '2023-08-24'}, {'titulo': 'Gaturro Messi', 'url': 'https://www.cuspide.com//Libro/9789878151885/Gaturro+Messi', 'AR$': '3690.00', 'U$s': '10.10', 'U$s_blue': 5.09, 'Fecha': '2023-08-24'}, {'titulo': 'La Scaloneta', 'url': 'https://www.cuspide.com//Libro/9789878151632/La+Scaloneta', 'AR$': '5690.00', 'U$s': '15.57', 'U$s_blue': 7.85, 'Fecha': '2023-08-24'}, {'titulo': 'Messi , Campeon Del Mundo', 'url': 'https://www.

Convierto la lista libros_completos en un dataframe, para posteriormente pasarlo a archivos csv

In [49]:
import pandas as pd

# Crear un DataFrame a partir de la lista de diccionarios
df = pd.DataFrame(libros_completo)

# Nombre del archivo CSV
nombre_archivo = "webscrapinglibros.csv"

# Guardar el DataFrame en un archivo CSV
df.to_csv(nombre_archivo, index=False, sep=';')